In [18]:
import math
import numpy as np

In [19]:
# Assuming data format: {(x, y, z): 'block_type'}
m = {}  # input map
graph = {}
stateful_graph = {}
Fsafe = 3
jump_height = 1 

In [20]:
world_txt_filename = 'skyblock/world-dump.txt'

In [21]:
with open(world_txt_filename, 'r') as file:
    lines = file.readlines()

In [22]:
for line in lines:
    if line.startswith('b'):
        parts = line.split() 
        x, z, y = map(int, parts[1:4]) 
        block_type = parts[4] 
        m[(x, y, z)] = block_type


In [23]:
def add_edge(x1, y1, z1, x2, y2, z2):
    distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)
    graph.setdefault((x1, y1, z1), []).append(((x2, y2, z2), distance))
    
def is_reachable(x, y, z):
    return m.get((x, y, z)) == 'solid' and m.get((x, y, z + 1)) != 'solid'

In [24]:
def construct_graph():
    for (x, y, z), block_type in m.items():
        if not is_reachable(x, y, z):
            continue
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]: # we only consider moving 4 dirs
            for dz in range(jump_height, -Fsafe - 1, -1):
                nx, ny, nz = x + dx, y + dy, z + dz
                if nz > z and not is_reachable(x, y, nz - 1):
                    continue
                if nz < z and not is_reachable(nx, ny, nz): # blocked by adjacent block
                    continue
                if is_reachable(nx, ny, nz):
                    add_edge(x, y, z, nx, ny, nz)
                    break

In [25]:
construct_graph()

In [26]:
graph

{(0, 1, 60): [((1, 1, 60), 1.0), ((0, 2, 60), 1.0)],
 (0, 2, 60): [((1, 2, 60), 1.0), ((0, 1, 60), 1.0)],
 (1, 0, 60): [((2, 0, 60), 1.0), ((1, 1, 60), 1.0)],
 (1, 1, 60): [((2, 1, 60), 1.0),
  ((0, 1, 60), 1.0),
  ((1, 2, 60), 1.0),
  ((1, 0, 60), 1.0)],
 (1, 2, 60): [((2, 2, 60), 1.0), ((0, 2, 60), 1.0), ((1, 1, 60), 1.0)],
 (2, 0, 60): [((3, 0, 60), 1.0), ((1, 0, 60), 1.0), ((2, 1, 60), 1.0)],
 (2, 1, 60): [((3, 1, 60), 1.0),
  ((1, 1, 60), 1.0),
  ((2, 2, 60), 1.0),
  ((2, 0, 60), 1.0)],
 (2, 2, 60): [((3, 2, 60), 1.0), ((1, 2, 60), 1.0), ((2, 1, 60), 1.0)],
 (3, 0, 60): [((4, 0, 60), 1.0), ((2, 0, 60), 1.0), ((3, 1, 60), 1.0)],
 (3, 1, 60): [((4, 1, 60), 1.0),
  ((2, 1, 60), 1.0),
  ((3, 2, 60), 1.0),
  ((3, 0, 60), 1.0)],
 (3, 2, 60): [((4, 2, 60), 1.0), ((2, 2, 60), 1.0), ((3, 1, 60), 1.0)],
 (4, 0, 60): [((5, 0, 60), 1.0), ((3, 0, 60), 1.0), ((4, 1, 60), 1.0)],
 (4, 1, 60): [((5, 1, 60), 1.0),
  ((3, 1, 60), 1.0),
  ((4, 2, 60), 1.0),
  ((4, 0, 60), 1.0)],
 (4, 2, 60): [((5, 2,

In [27]:
def add_edge_with_resource(x1, y1, z1, k1, x2, y2, z2, k2, ex_cost):
    distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)
    stateful_graph.setdefault((x1, y1, z1, k1), []).append(((x2, y2, z2, k2), distance + ex_cost))

def construct_graph_with_resources(K):
    for (x, y, z), block_type in m.items():
        for k in range(K+1):  
            if block_type == 'solid':
                for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                    for dz in [1]: # can only destruct block at z+1
                        nx, ny, nz = x + dx, y + dy, z + dz
                        # no block or not solid or have resource
                        if (nx, ny, nz) not in m or (m[(nx, ny, nz)] != 'solid' or k > 0):
                            nk = k if m.get((nx, ny, nz)) != 'solid' else k-1
                            add_edge_with_resource(x, y, z, k, nx, ny, nz-1, nk, 1) 
            else:
                for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                    for dz in range(jump_height, -Fsafe - 1, -1):
                        nx, ny, nz = x + dx, y + dy, z + dz
                        if (nx, ny, nz) in m and k >= 0:
                            nk = k  # No resource used
                            add_edge_with_resource(x, y, z, k, nx, ny, nz, nk, 1) 
                            break


In [30]:
K = 3
construct_graph_with_resources(K)

In [31]:
stateful_graph

{(0, 0, 58, 0): [((-1, 0, 58, 0), 2.0), ((0, -1, 58, 0), 2.0)],
 (0, 0, 58, 1): [((1, 0, 58, 0), 2.0),
  ((-1, 0, 58, 1), 2.0),
  ((0, 1, 58, 0), 2.0),
  ((0, -1, 58, 1), 2.0)],
 (0, 0, 58, 2): [((1, 0, 58, 1), 2.0),
  ((-1, 0, 58, 2), 2.0),
  ((0, 1, 58, 1), 2.0),
  ((0, -1, 58, 2), 2.0)],
 (0, 0, 58, 3): [((1, 0, 58, 2), 2.0),
  ((-1, 0, 58, 3), 2.0),
  ((0, 1, 58, 2), 2.0),
  ((0, -1, 58, 3), 2.0)],
 (0, 1, 58, 0): [((-1, 1, 58, 0), 2.0)],
 (0, 1, 58, 1): [((1, 1, 58, 0), 2.0),
  ((-1, 1, 58, 1), 2.0),
  ((0, 2, 58, 0), 2.0),
  ((0, 0, 58, 0), 2.0)],
 (0, 1, 58, 2): [((1, 1, 58, 1), 2.0),
  ((-1, 1, 58, 2), 2.0),
  ((0, 2, 58, 1), 2.0),
  ((0, 0, 58, 1), 2.0)],
 (0, 1, 58, 3): [((1, 1, 58, 2), 2.0),
  ((-1, 1, 58, 3), 2.0),
  ((0, 2, 58, 2), 2.0),
  ((0, 0, 58, 2), 2.0)],
 (0, 2, 58, 0): [((-1, 2, 58, 0), 2.0), ((0, 3, 58, 0), 2.0)],
 (0, 2, 58, 1): [((1, 2, 58, 0), 2.0),
  ((-1, 2, 58, 1), 2.0),
  ((0, 3, 58, 1), 2.0),
  ((0, 1, 58, 0), 2.0)],
 (0, 2, 58, 2): [((1, 2, 58, 1), 2.0),

In [ ]:
# TODO: below are modified based on chatgpt's solution

import math
from queue import PriorityQueue

# Assuming graph is already defined as shown in previous examples

def euclidean_distance(x1, y1, z1, x2, y2, z2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)

def a_star_search(start, goal):
    open_set = PriorityQueue()
    open_set.put((0, start))
    
    came_from = {}
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0
    
    f_score = {node: float('inf') for node in graph}
    f_score[start] = euclidean_distance(*start, *goal)
    
    while not open_set.empty():
        current = open_set.get()[1]  # Get node with lowest f_score value
        
        if current == goal:
            path = [current]
            total_cost = g_score[current]  # Total cost to reach the goal
            while current in came_from:
                current = came_from[current]
                path.append(current)
            path.reverse()
            return path, total_cost
        
        for neighbor, distance in graph[current]:
            tentative_g_score = g_score[current] + distance
            
            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + euclidean_distance(*neighbor, *goal)
                open_set.put((f_score[neighbor], neighbor))
    
    return False, 0 

# Example usage
# start = (0, 0, 0)  # Assuming start position
# goal = (5, 5, 5)  # Assum
# path, cost = a_star_search(start, goal)
# print("Path:", path)
# print("Total cost:", cost)


In [ ]:
def a_star_search_with_resources(start, goal, K):
    open_set = PriorityQueue()
    open_set.put((0, start + (K,)))  # Start node now includes initial resources
    
    came_from = {}
    g_score = {node: float('inf') for node in graph}
    g_score[start + (K,)] = 0
    
    f_score = {node: float('inf') for node in graph}
    f_score[start + (K,)] = euclidean_distance(*start, *goal)
    
    while not open_set.empty():
        current = open_set.get()[1]
        
        if current[:3] == goal:
            path = []
            while current in came_from:
                path.append(current[:3])
                current = came_from[current]
            path.append(start)
            return path[::-1]
        
        for (neighbor, k_rest), distance in graph[current]:
            tentative_g_score = g_score[current] + distance
            
            if tentative_g_score < g_score[(neighbor, k_rest)]:
                came_from[(neighbor, k_rest)] = current
                g_score[(neighbor, k_rest)] = tentative_g_score
                f_score[(neighbor, k_rest)] = tentative_g_score + euclidean_distance(*neighbor, *goal)
                open_set.put((f_score[(neighbor, k_rest)], (neighbor, k_rest)))
    
    return False
